# Childrens Act Disposals

In [2]:
# libraries
import pandas as pd
import pydbtools as pydb
import boto3
from datetime import datetime
#import re

In [3]:
# Structured  dataframes
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

In [4]:
# Assigning databases and S3 bucket
db1 = "familyman_dev_v2"
fcsq_db = "fcsq"
s3 = boto3.resource("s3")
bucket = s3.Bucket("alpha-family-data")
snapshot_date = "2021-08-19"

### Extracting disposals 

In [5]:
# creating table childrens_act_disposals afresh.
d1 = f""" 
SELECT 
  event,
  case_number,
  event_model,
  receipt_date,
  entry_date,
  error,
  creating_court,
  CAST(NULL AS VARCHAR) AS field_model,
  CAST(NULL AS VARCHAR) AS value
FROM 
  {db1}.events 
WHERE 
  ((event_model IN ('C23','C25','C26','C27','C28','C29',
                        'C30','C31','C32A','C32B','C33','C34A','C34B','C35A','C35B','C37',
                        'C38A','C38B','C39','C40','C42','C43A','C44A','C44B','C45A','C45B',
                        'C53','C80','C81','C82') 
            AND error = 'N')
    OR (event_model = 'CPA' AND error = 'N' AND narrative Is Null))
  AND mojap_snapshot_date = date '{snapshot_date}'


UNION ALL


SELECT 
  e.event,
  e.case_number,
  e.event_model,
  e.receipt_date,
  e.entry_date,
  e.error,
  e.creating_court,
  f.field_model,
  f.value
FROM 
  {db1}.events e
  INNER JOIN {db1}.event_fields f
    ON e.event = f.event
WHERE 
  ((f.field_model IN ('C21_3', 'C24_6', 'C36_5', 'C43_5', 'ORDNOM_5', 'ORDREF_5'))
    OR error = 'N')
  AND e.mojap_snapshot_date = date '{snapshot_date}'
  AND f.mojap_snapshot_date = date '{snapshot_date}'
  
"""

pydb.create_temp_table (d1,"ca_disps")

In [ ]:
pydb.read_sql_query ("select * from __temp__.ca_disps")